# Introduction

In the past couple notebooks we explored the data after it was cleaned. We found some interesting things regarding how COVID has affected airline flight operations since the start of 2020. Now we will look at how Python helps you to reformat datasets and combine them with other datasets in order to enhance your analysis.

# Import and Ingest

In [ ]:
# Import Libraries

import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import math

Let's start by importing our dataset from last time: eda_data.csv.

In [ ]:
# Answer

filepath = r'/home/jovyan/eda_data.csv'
df = pd.read_csv(filepath)

#### Which airline tends to operate the shorter flights?

In order to answer this question, we will need to have information that cannot be easily discerned from the current data format. Each record has only a single time column which records when that specific takeoff or landing event happened. In order to compute the duration of a flight, we need to know both the takeoff and landing times for a single flight. Luckily, both of these observations are in our dataset, but they take a little work to match. 

The following code will do it for you. 

In [ ]:
# Sorting by Time so that we can match the flight records
df['time'] = pd.to_datetime(df['time'])
df_flights = df.sort_values(['call_sign','time'])

# Creating additional columns to match flight records. There should be consecutive off and on records for a given flights on
# a given route. .shift(1) moves every value down one row to combine data from consecutive records
df_flights['call_sign_shift'] = df_flights['call_sign'].shift(1)
df_flights['event_shift'] = df_flights['event'].shift(1)
df_flights['airport_shift'] = df_flights['airport'].shift(1)
df_flights['time_shift'] = df_flights['time'].shift(1)

# Filtering to the records that math consecutive off and on events for the same call sign 
df_flights_complete = df_flights.loc[(df_flights['event'] == 'on') &
                                (df_flights['event_shift'] == 'off') & 
                                (df_flights['call_sign'] == df_flights['call_sign_shift']) & 
                                (df_flights['airport_shift'] == df_flights['departure_airport'])].copy()


In [ ]:
df_flights_complete.head()

We now have records with matching flight numbers, origin, and destination and corresponding off and on events and times. Let's compute the trip duration and drop the extra fields that were created. Note that by requiring that we have both an off and on event for a flight, we are naturally restricting the dataset to only domestic flights that both depart from and arrive at one of the 34 airports included in the dataset.

In [ ]:
df_flights_complete.loc[:, 'length_of_trip'] = (df_flights_complete['time'].sub(df_flights_complete['time_shift'])) / np.timedelta64(1, 'm')

# Dropping all unnecessary columns 
df_flights_complete = df_flights_complete.drop(columns = ['call_sign_shift','event_shift','airport_shift','time_shift', 'type_of_flight'])

Let's check the distribution of flight durations for the different airlines to make a quick check that everything is done right. We'll use the *.describe()* method to summarize the distribution. This will give us the mean, standard deviation, minimum, and maximum values for the trip duration along with the 25th, 50th (median), and 75th percentile values.

In [ ]:
df_flights_complete['length_of_trip'].describe()

Trip duration is expressed in minutes, so this is telling us that the median flight duration is about 2 hours, and 75% of flight durations are under 3 hours. This seems reasonable. But look at the minimum and maximum values: there is at least one flight that was measured at less than 8 minutes, and another flight that was measured at over 100,000 minutes - 10 weeks! This clearly means something funky is going on in the data so let's look into it.

Use the *.sort()* and *.head()* methods to get a look at the records with the 10 longest and shortest durations. Note: if you don't want to change the direction of the sort, you can also use the *.tail()* method in place of *.head()* to grab the last rows of a data frame.

In [ ]:
# Answer - part 1
df_flights_complete.sort_values('length_of_trip').head()

In [ ]:
# Answer - part 2
df_flights_complete.sort_values('length_of_trip').tail()

First: the short flights. These all appear to be between either the same airport (KJFK-KJFK) or nearby airports in the same city (such as KMDW (Midway) and KORD (O'Hare) in Chicago). It is possible that these are real flights - equipment problems during takeoff sometimes result in a plan immediately returning to the airport or diverting to a nearby airport. Even if this is the case, these are not the types of flights that we are interested in. A reasonable lower bound for the duration of a commercial flight traveling between major airports is 45 minutes.

For the longest flights, these look like legitimate city pairs, but the durations are unrealistically long. This could be errors in the source data or in our logic for matching off and on events. Either way, we do not want these bogus records to skew our results. A typical flight from Seattle, WA to Miami, Fl takes about 6 hours, or 360 minutes. An upper bound of 480 minutes (8 hours) for a flight within the contental United States seems reasonable.

Let's filter our flight lists to include only flight durations between 45 and 480 minutes. While we're at it, let's remove cases where departure airport and destination airport are the same just in case.

In [ ]:
# Answer

index = (df_flights_complete['length_of_trip'] <= 480) & (df_flights_complete['length_of_trip'] >= 45) & \
  (df_flights_complete['departure_airport'] != df_flights_complete['destination_airport'])

df_flight_clean = df_flights_complete[index]

Let's take another look at the distribution of flight times. This time, we'll use a type of graph called a histogram. This is similar to a bar chart, but is used to plot the number of observations that fall into different ranges along an axis. The heights of the bars allow you to quickly assess how flight durations are spread between 45 and 480 minutes.

In [ ]:
plt.hist(df_flight_clean['length_of_trip'], edgecolor = 'black', bins = 30)
plt.title("Distribution of Flight Durations")
plt.xlabel('Flight duration (minutes)')
plt.ylabel('# of Flights')
plt.show()

This seems reasonable. Most flights last less than 3 hours, with longer haul flights less frequent and tapering off beyond 6 hours. Note the *bins* parameter in the plotting function above. This specifies the number of bars that will be used in the histogram. The number of bins can have a significant influence on the apparent shape of the data. Try recreating the above plot with various bin counts from 5 to 100 to see how your interpretation of the data changes.

Now let's look at duration distributions for each airline. For this comparison, we will use a plot called a boxplot also sometimes known as a box and whisker plot. The box shows the primary range of the data (25th to 75th percentiles) with a line in the middle for a median value. The "whiskers" that stretch out on the top and the bottoms show an extended range of the data (we'll leave it to you to look up the math if you are interested). Any data points that fall outside of the extended range are flagged as outliers and are plotted as individual data points.

In [ ]:
airline_aal = df_flight_clean[df_flight_clean['airline'] == 'AAL']['length_of_trip']
airline_dal = df_flight_clean[df_flight_clean['airline'] == 'DAL']['length_of_trip']
airline_swa = df_flight_clean[df_flight_clean['airline'] == 'SWA']['length_of_trip']
airline_ual = df_flight_clean[df_flight_clean['airline'] == 'UAL']['length_of_trip']

airline_boxplot = np.array([airline_aal, airline_dal, airline_swa, airline_ual], dtype=object)

plt.boxplot(airline_boxplot, labels = ['American','Delta','Southwest','United'], vert = False)
plt.title("Distribution of Flight Durations by Airline")
plt.xlabel('Flight duration (minutes)')
plt.show()

Looking at boxes, we can see that, in general, Southwest flights seem to be of shorter duration than other airlines. In particular, 75 percent of Southwest flights are 150 minutes or less, which is 30-45 minutes shorter than the 75th percentile duration on other airlines.

The above boxplot compares actual flight durations, but what if we wanted to know how airlines fare relative to typical flight times on a given route? Are certain airlines likely to fly a route faster than others?

Let's start by calculating the average flight duration for each route in the dataset and taking a quick look at the structure of this new dataset.

In [ ]:
df_avg_flight = df_flight_clean.groupby(['destination_airport','departure_airport'])['length_of_trip'].mean().reset_index(
    name = 'avg_flight_length')
df_avg_flight.head()

Now we need to add the averages to the corresponding individual flight records to be able to compare flight durations against the route average. This requires us to merge the datasets. 

You have might seen a merge statement in some of the earlier notebooks, but here we will explain how it is done using the *.merge()* method. The basic idea is that you select one or more corresponding between the two data sets (departure and destination airport, in our case), the merge will match rows from the first dataset with rows from the second dataset based on the selected rows, and the data from the matching rows will be combined into one longer row. 

In our case, we will merge based on the destination and departure airports. When a flight records and an average duration record have matching departure airports and matching destination airports then the average duration for the route will be added to the end of the flight record. In the below code, the columns passed to the "on" argument in the merge method will be used for matching.

In [ ]:
df_flights_with_avg = df_flight_clean.merge(df_avg_flight, on = ['destination_airport','departure_airport'])

df_flights_with_avg.head()

Now that we have the average route duration merged in, we can compute the difference between a flight duration and its corresponding route average.

In [ ]:
df_flights_with_avg['adjusted_to_avg'] = df_flights_with_avg['length_of_trip'] - df_flights_with_avg['avg_flight_length']

In [ ]:
df_flights_with_avg.head()

So by looking at the dataset, we see that the column avg_flight_length appears on the new dataset. A positive value implies that the flight was longer than the route average, while a negative value implies that it was shorter.

Now let's look at the distribution of the flight durations as they compare to the average route flight duration.

In [ ]:
# Answer

airline_aal = df_flights_with_avg[df_flights_with_avg['airline'] == 'AAL']['adjusted_to_avg']
airline_dal = df_flights_with_avg[df_flights_with_avg['airline'] == 'DAL']['adjusted_to_avg']
airline_swa = df_flights_with_avg[df_flights_with_avg['airline'] == 'SWA']['adjusted_to_avg']
airline_ual = df_flights_with_avg[df_flights_with_avg['airline'] == 'UAL']['adjusted_to_avg']

airline_boxplot = np.array([airline_aal, airline_dal, airline_swa, airline_ual], dtype=object)

plt.boxplot(airline_boxplot, labels = ['American','Delta','Southwest','United'], vert = False)
plt.title("Deviation from Average Route Duration by Airline")
plt.xlabel('Deviation from average route duration (minutes)')
plt.show()

It is difficult to discern any difference between the main cluster of values around zero. However, there are some notable outliers with durations 2+ hours longer than the route average. Lets's take a closer look at a few of those.

In [ ]:
df_flights_with_avg[(df_flights_with_avg['airline'] == 'AAL') & (df_flights_with_avg['adjusted_to_avg'] > 150)]

We do not have much information to help understand these cases, but one clue: there were significant storms in Charlotte on February 6 that caused a number of air traffic problems, including flights having to temporarily divert to land and refuel at another airport while waiting for the weather to clear. Look at flight AAL2052 - a 7-hour flight from KORD (Chicago) to KCLT (Charlotte) on... February 6!

So perhaps some of these are accurate travel times (even if the flight was not in the air the entire time), but even so, they are not the typical flights that we are interested in for this analysis. So instead, let's turn to some basic summary stats to make our comparison.

In [ ]:
df_avg = df_flights_with_avg.groupby('airline')['adjusted_to_avg'].mean().reset_index(name = 'mean_relative_duration')
df_std = df_flights_with_avg.groupby('airline')['adjusted_to_avg'].std().reset_index(name = 'std_relative_duration')

df_best = df_avg.merge(df_std, on = 'airline')

In [ ]:
df_best

There is surprisingly little variation between airlines in terms of their mean duration relative to route average - all are well under a minute. So don't expect one airline to consistently get you to your destination 10 minutes ahead of any other. 

*std* is the standard deviation of this relative duration - a measure of how widely the value varies from one flight to the next. So an airline that is on average close to the route average duration could achieve that by always arriving within a few minutes of the route average (low standard deviation) or by behing 30 minutes ahead on half of its flights and 30 minutes behind on the other half (high standard deviation). By that metric, Delta and Southwest are slightly more consistent than American and United, but the difference is less than 2 minutes.

So all-in-all, you shouldn't expect one airline to beat the others on flight duration between takeoff and landing.

# Merging Data

We have been working with airports so far just based on their names - we're not using any data on the airports themselves. As it turns out, we have another set of data that can be merged with our existing set that might make exploring this data much more interesting. It is the latitude, longtitude and altitudes of the airports! This opens up a whole new realm of analysis: geospatial analysis that takes into account absolute and relative locations of locations, objects, or events.

Let's see if there are any gems when we include this dataset. 

In [ ]:
df_airport_desc = pd.read_csv(r'/home/jovyan/airport_data.csv')

In [ ]:
print(df_airport_desc.shape)
df_airport_desc.head()

Looks good so far, though 2.5M is a lot of rows for airport info. We'll revisit that later...

Since we're working with this data for the first time, we need to do a few quality checks to make sure it is clean. First let's check for missing values, and then we'll see if there are any duplicates in the airport column. Hint: We can use the *.duplicated()* method to check for duplicates. 

In [ ]:
# Answer

missing_answer = df_airport_desc.isnull().any()
print('Are there any missing values? \n', missing_answer)

dup_answer = df_airport_desc['airport'].duplicated().any()
print('\n Are there duplicates (multiple records for a single airport)?', dup_answer)

The *.any()* method is a quick check to see if there are any true values in column or array. So we see that there are duplicates - that may explain the 2.5M rows - but no null values. Let's look at some of the duplicates.

In [ ]:
df_airport_desc[df_airport_desc['airport'] == 'KMIA'].head()

It seems like there are many latitudes, longitudes and altitudes for each airport, though all fairly close to one another. We can average them to get fairly good estimate of airport location. 

In [ ]:
# Answer

df_avg_airport = df_airport_desc.groupby('airport').mean().reset_index()

In [ ]:
print(df_avg_airport.shape)
df_avg_airport.head()

This seems more like what was expected.

Now that we have a general idea of the longitudes, latitudes and altituedes, let's do something cool and plot them on a map so we can see all the various airports. We will not go into the details of the logic for building these charts, but feel free to dig in on your own.

In [ ]:
fig = go.Figure(data=go.Scattergeo(
        lon = df_avg_airport['longitude'],
        lat = df_avg_airport['latitude'],
        text = df_avg_airport['airport'],
        mode = 'markers'
        ))

fig.update_layout(
        geo_scope='usa'
    )
fig.show()

The 42 airports are mostly in the continental US with Honolulu, Hawaii as the single exception. By itself, the data is not all that useful; however, we combine it with our previous data set, we can start putting our flight data in geographical context. So let's merge these two datasets togethers.

We will use the flight data from earlier in the notebook. We will have to merge the data sets together twice: once to add coordinates for departure airports, and a second time for destination airports.

We have provided the skeleton of a merge here for you to populate. This merge will be a little different from the one from above. Some parameters have changed from the first merge:

* *right_on* and *left_on*: when the column names to be matched are different between the two datasets, we use these arguments to denote the columns to be matched from each dataset separately
* *how*: the 'left' option says not to throw away any rows from the first dataset (flights), even if they do not have a match in the second data set (airports).
* *suffixes*: these will be appended to the column names from each dataset to track which dataset they came from.

In [ ]:
# Answer

df_combine_departure = df_flights_with_avg.merge(df_avg_airport, right_on = ['airport'], left_on = ['departure_airport'], how = 'left', suffixes=('_original', '_departure'))
df_combine_full = df_combine_departure.merge(df_avg_airport, right_on = ['airport'], left_on = ['destination_airport'], how = 'left', suffixes=('_departure', '_destination'))


Let's look at the column names from the merged dataset.

In [ ]:
df_combine_full.columns

Let's clean up unneeded columns before proceeding.

In [ ]:
df_combine_clean = df_combine_full.drop(columns = ['airport_departure','event','call_sign','airport_original','airport'])

With the coordinates for both the departure and destination airports, we can now compute approximate route distances based on the great circle distance between airports. We have provided a function that will calculate the distance between two latitudes and longitudes in kilometers. 

In [ ]:
def distance_calculator(lat1_d,lon1_d,lat2_d,lon2_d):
    # approximate radius of earth in km
    R = 6373.0
    
    lat1 = math.radians(lat1_d)
    lon1 = math.radians(lon1_d)
    lat2 = math.radians(lat2_d)
    lon2 = math.radians(lon2_d)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    
    return distance

Let's apply this function to our merged data to compute the route distance for each flight. The lambda notation below is a way of applying a function to each row of a dataset - don't worry if you don't follow the syntax.

In [ ]:
df_combine_clean['distance'] = df_combine_clean.apply(lambda x: distance_calculator(x['latitude_departure'],x['longitude_departure'],x['latitude_destination'],x['longitude_destination']), axis = 1)

Let's look at the distribution route distances. First we'll look at the distribution for unique routes - i.e., ignoring the number of flights flown on each route. Grab the unique departure, destination, distance triplets.

In [ ]:
# Answer 

df_dedup = df_combine_clean[['departure_airport','destination_airport','distance']].drop_duplicates()

Now let's look at the distribution of the distances across routes. Hint: Try using a histogram.

In [ ]:
# Answer

plt.hist(df_dedup['distance'], edgecolor = 'black', bins = 30);
plt.title('Distribution of Route Distances')
plt.xlabel('Distance (km)')
plt.ylabel('Route count')
plt.show();

We see that most of the airports are roughly ~1500 kms from each other. There are few routes that are really long. They are probably the cross country flights. Let's take a look at them. 

In [ ]:
# Answer
long_flights = df_dedup[df_dedup['distance'] > 4000].sort_values('distance', ascending = False).head(6)
long_flights

As expected, the longest routes are transcontinental, and in particular the diagonal routes that cut between northeast and southwest or northwest and southeast.

Now that we have found that the longest routes, let's jump back up to the flight level and see how the flights are distributed by route distance. Try another histogram.

In [ ]:
# Answer

plt.hist(df_combine_clean['distance'], edgecolor = 'black', bins = 30);
plt.title('Distribution of Flight Distances')
plt.xlabel('Distance (km)')
plt.ylabel('Flight count')
plt.show();

Compared to the route distance distribution, the shorter routes (under 2000km) cover an even higher proportion of flight distances. This implies that not only are there more flights flying shorter routes, but each shorter route has a higher density of flights as well.

As one final exercise, let's compare the flight distances by airline. Use whatever method you prefer.

In [ ]:
# Answer 1

fast = df_combine_clean.groupby('airline')['distance'].mean().reset_index()
fast

In [ ]:
# Answer 2
airline_aal = df_combine_clean[df_combine_clean['airline'] == 'AAL']['distance']
airline_dal = df_combine_clean[df_combine_clean['airline'] == 'DAL']['distance']
airline_swa = df_combine_clean[df_combine_clean['airline'] == 'SWA']['distance']
airline_ual = df_combine_clean[df_combine_clean['airline'] == 'UAL']['distance']

airline_boxplot = np.array([airline_aal, airline_dal, airline_swa, airline_ual], dtype=object)

plt.boxplot(airline_boxplot, labels = ['American','Delta','Southwest','United'], vert = False)
plt.title("Distribution of Flight Distances by Airline")
plt.xlabel('Flight distance (km)')
plt.show()

In general, it appears that Southwest tends to fly shorter domestic routes while United tends to fly longer routes.

# Conlusion

At this point, we have looked at this dataset and found a lot of interesting things. We have covered all of the key steps of data analysis - importing, cleaning, and analyzing the data - and hopefully you are starting to see the power and flexibility that a tool like Python provides every step along that path. The next step is to come up with some of your own analysis questions and to get your hands dirty figuring out how to answer them with the data that we have provided. In the last notebook, we will give you a space to do this. 